In [1]:
import numpy as np
import librosa
import sklearn
import pandas
import python_speech_features
import os
import scipy.io.wavfile as wav
from audiolazy import lazy_lpc
import pandas as pd
from sklearn.svm import SVC
from sklearn.utils import shuffle

In [2]:
subdirectories=os.listdir(os.getcwd() + "/genres")
subdirectories

['blues',
 'classical',
 'country',
 'disco',
 'hiphop',
 'jazz',
 'metal',
 'pop',
 'reggae',
 'rock']

In [3]:
path_of_audios=[]
lables=[]
len_of_each_genre=[]
prev=0
for subdirectory in subdirectories:
    path_of_audio=librosa.util.find_files(os.getcwd() + "/genres" + "/" + subdirectory)
    path_of_audios+=path_of_audio
    l=[subdirectory]*(len(path_of_audios)-prev)
    prev=len(path_of_audios)
    lables+=l
    len_of_each_genre.append(len(l))


In [4]:
len_of_each_genre

[100, 100, 100, 100, 100, 100, 100, 100, 100, 100]

In [5]:
len(lables)

1000

In [6]:
len(path_of_audios)

1000

In [7]:
audios=[]
for path_of_audio in path_of_audios:
    signal,sampling_rate=librosa.load(path_of_audio,duration=5.0)
    audios.append(signal)

sampling_rate
    

22050

In [8]:
type(audios)

list

In [9]:
len(audios)

1000

In [10]:
audios=np.array(audios)
audios.shape

(1000, 110250)

In [11]:
feature_vector=[]

In [12]:
zero_crossing_rate=librosa.feature.zero_crossing_rate(y=audios[0])
zero_crossing_rate.shape

(1, 216)

In [13]:
zero_crossings_rates=[]
for audio in audios:
    zero_crossings_rate=librosa.feature.zero_crossing_rate(y=audio)
    zero_crossings_rates.append(zero_crossings_rate[0])
    

In [14]:
zero_crossings_rates=np.array(zero_crossings_rates)
zero_crossings_rates.shape

(1000, 216)

In [15]:
zero_crossings_rates[0]

array([0.06494141, 0.08300781, 0.10791016, 0.10351562, 0.09423828,
       0.08935547, 0.08447266, 0.08447266, 0.08300781, 0.09082031,
       0.09619141, 0.08251953, 0.07324219, 0.07421875, 0.07421875,
       0.08496094, 0.10107422, 0.10986328, 0.11132812, 0.10302734,
       0.09326172, 0.08496094, 0.0703125 , 0.07177734, 0.06591797,
       0.05761719, 0.05761719, 0.05126953, 0.05273438, 0.0625    ,
       0.07324219, 0.08398438, 0.09472656, 0.09082031, 0.08691406,
       0.08007812, 0.07568359, 0.07373047, 0.06689453, 0.06347656,
       0.05126953, 0.04736328, 0.0546875 , 0.05273438, 0.06445312,
       0.06542969, 0.06689453, 0.07714844, 0.0703125 , 0.07128906,
       0.07568359, 0.07324219, 0.07373047, 0.06884766, 0.0625    ,
       0.06152344, 0.0625    , 0.07128906, 0.07177734, 0.07421875,
       0.07373047, 0.06640625, 0.06835938, 0.06982422, 0.06494141,
       0.06494141, 0.06542969, 0.05175781, 0.05078125, 0.0546875 ,
       0.06884766, 0.09130859, 0.10400391, 0.11376953, 0.10205

In [16]:
mfcc=librosa.feature.mfcc(y=audios[0],n_mfcc=13)
mfcc.shape

(13, 216)

In [17]:
mfccs=[]
for audio in audios:
    mfcc=librosa.feature.mfcc(y=audio,n_mfcc=13)
    mfccs.append(mfcc)
mfccs=np.array(mfccs)
mfccs.shape


(1000, 13, 216)

In [35]:
mfcc_deltas=[]
for audio in audios:
    mfcc=librosa.feature.mfcc(y=audio,n_mfcc=13)
    mfcc_delta=librosa.feature.delta(mfcc)
    mfcc_deltas.append(mfcc_delta)

mfcc_deltas=np.array(mfcc_deltas)
mfcc_deltas.shape

(1000, 13, 216)

In [18]:
spectral_rolloffs=[]
for audio in audios:
    spectral_rolloff=librosa.feature.spectral_rolloff(y=audio)
    spectral_rolloffs.append(spectral_rolloff[0])
spectral_rolloffs=np.array(spectral_rolloffs)
spectral_rolloffs.shape

(1000, 216)

In [19]:
spectral_centroids=[]
for audio in audios:
    spectral_centroid=librosa.feature.spectral_centroid(y=audio)
    spectral_centroids.append(spectral_centroid[0])

spectral_centroids=np.array(spectral_centroids)
spectral_centroids.shape

(1000, 216)

In [166]:
beat_features=[]
tempos=[]
for audio in audios:
    y_harmonic, y_percussive = librosa.effects.hpss(audio)
    mfcc=librosa.feature.mfcc(y=audio,n_mfcc=13)
    mfcc_delta=librosa.feature.delta(mfcc)
    tempo, beat_frames = librosa.beat.beat_track(y=audio)
    tempos.append(tempo)
    beat_mfcc_delta = librosa.util.sync(np.vstack([mfcc, mfcc_delta]),
                                    beat_frames)
    chromagram = librosa.feature.chroma_cqt(y=y_harmonic)
    beat_chroma = librosa.util.sync(chromagram,
                                beat_frames,
                                aggregate=np.median)
    beat_feature = np.vstack([beat_chroma, beat_mfcc_delta])
    beat_features.append(beat_feature)

tempos=np.array(tempos)
print(tempos.shape)
final_beat_features=[]
for i in range(len(tempos)):
    features=[]
    for j in range(38):
        features.append(np.array(beat_features[i][j]).mean())
    final_beat_features.append(features)
    
final_beat_features=np.array(final_beat_features)
final_beat_features.shape
        

        

(1000,)


(1000, 38)

In [68]:
spectral_bandwidths=[]
for audio in audios:
    spectral_bandwidth=librosa.feature.spectral_bandwidth(audio)
    spectral_bandwidths.append(spectral_bandwidth[0])

spectral_bandwidths=np.array(spectral_bandwidths)
spectral_bandwidths.shape

(1000, 216)

In [122]:
spectral_bandwidths[0]

array([2340.92967734, 2268.36012036, 2308.61530036, 2204.22709242,
       2089.44630245, 2122.57154111, 2086.64967829, 1973.58755412,
       1957.04117619, 1884.24053067, 1803.83264329, 1783.97930014,
       1645.50249281, 1547.61989131, 1514.77276836, 1461.26894281,
       1425.41440512, 1470.77787391, 1597.16259893, 1803.20511131,
       1865.48670881, 1898.55825723, 1852.39264327, 1928.21755766,
       2112.5221518 , 2070.41613948, 1938.11477974, 1853.24542712,
       1918.84916396, 1999.0888763 , 2273.46751914, 2343.41669456,
       2159.451153  , 2036.1221809 , 2006.78874013, 1966.42524106,
       1948.9096993 , 1985.66728702, 1985.35844271, 2011.70470277,
       1988.35922154, 2059.14472218, 2094.25329776, 1939.42300426,
       2042.37984037, 2367.96957517, 2341.24411188, 2220.33380166,
       2027.48690166, 1950.27654696, 1907.4659736 , 2009.53455828,
       2104.31902135, 2017.21280756, 2019.09432717, 2014.67383073,
       1964.91264561, 2026.74656652, 2045.36762667, 2018.21257

In [132]:
chroma_features=[]
for audio in audios:
    y_harmonic, y_percussive = librosa.effects.hpss(audio)
    tempo, beat_frames = librosa.beat.beat_track(y=y_percussive)
    mfcc = librosa.feature.mfcc(y=audio)
    mfcc_delta = librosa.feature.delta(mfcc)
    beat_mfcc_delta = librosa.util.sync(np.vstack([mfcc, mfcc_delta]),
                                    beat_frames)
    chromagram = librosa.feature.chroma_cqt(y=y_harmonic)
    chroma_features.append(chromagram)
    

chroma_features.shape
    
    

AttributeError: 'list' object has no attribute 'shape'

In [137]:
chroma_features=np.array(chroma_features)
chroma_features.shape

(1000, 12, 216)

In [153]:
feature_vector=[]

for i in range(len(audios)):
    single_feature=[]
    single_feature.append(zero_crossings_rates[i].mean())
    single_feature.append(zero_crossings_rates[i].std())
    single_feature.append(spectral_rolloffs[i].mean())
    single_feature.append(spectral_rolloffs[i].std())
    single_feature.append(spectral_centroids[i].mean())
    single_feature.append(spectral_centroids[i].std())
    single_feature.append(spectral_bandwidths[i].mean())
    single_feature.append(spectral_bandwidths[i].std())
    #single_feature.append(tempos[i])
    for j in range(13):
        single_feature.append(mfccs[i,j].mean())
        single_feature.append(mfccs[i,j].std())
        single_feature.append(mfcc_deltas[i,j].mean())
        single_feature.append(mfcc_deltas[i,j].std())
 #   for k in range(38):
  #      single_feature.append(final_beat_features[i,k])
    for k in range(12):
        single_feature.append(chroma_features[i,k].mean())
        single_feature.append(chroma_features[i,k].std())
        
    
    feature_vector.append(single_feature)
        

feature_vector=np.array(feature_vector)
feature_vector.shape

(1000, 84)

In [154]:
feature_vector

array([[8.39527271e-02, 2.87213936e-02, 3.74433492e+03, ...,
        2.41385180e-01, 4.45198596e-01, 2.49997572e-01],
       [4.53129521e-02, 3.49450449e-02, 3.35364685e+03, ...,
        2.99553477e-01, 5.85742739e-01, 2.83669027e-01],
       [7.24803783e-02, 5.33338387e-02, 3.03423767e+03, ...,
        2.47582115e-01, 5.47189817e-01, 3.02046575e-01],
       ...,
       [7.26747866e-02, 2.29757369e-02, 3.32648112e+03, ...,
        2.10903251e-01, 6.88286276e-01, 2.89588316e-01],
       [3.11618381e-02, 8.85275277e-03, 2.66139425e+03, ...,
        2.15756177e-01, 3.67021938e-01, 2.18490675e-01],
       [6.51380751e-02, 2.03440201e-02, 2.72544556e+03, ...,
        2.03375617e-01, 5.17884008e-01, 4.01487466e-01]])

In [155]:
scaler = sklearn.preprocessing.MinMaxScaler(feature_range=(-1, 1))
feature_vector=scaler.fit_transform(feature_vector)
dataset_pandas=pd.DataFrame(feature_vector)
dataset_pandas['genre']=lables
dataset_pandas.to_csv("data_set.csv", index=False)



In [156]:
dataset_pandas.shape

(1000, 85)

In [157]:
data=pd.read_csv('data_set.csv')

In [158]:
data=shuffle(data)

In [159]:
data.iloc[0]

0         -0.807148
1         -0.905766
2        -0.0277576
3         -0.474463
4         -0.407554
5         -0.663719
6         0.0525614
7         -0.654475
8          0.331864
9          -0.77855
10        -0.193457
11        -0.633633
12        -0.176485
13         -0.75328
14        -0.116579
15         -0.61373
16         0.314786
17         -0.76307
18        0.0820559
19        -0.713584
20         0.959166
21        -0.886056
22        -0.136498
23        -0.771452
24         0.124193
25        -0.903113
26         0.105833
27         -0.68249
28         0.730549
29        -0.837039
            ...    
55         -0.57685
56         0.139025
57        -0.705473
58         -0.32712
59        -0.207706
60        -0.149622
61         0.271759
62         -0.27985
63        -0.111471
64        -0.397814
65         -0.15696
66        -0.220838
67         0.190553
68        -0.399679
69        -0.157826
70        -0.259626
71         -0.19224
72       -0.0954993
73        0.0883845


In [160]:
data_set_np=np.array(data)
X = data_set_np[:,:-1]
y = data_set_np[:,-1]

In [161]:
train_x[0]
train_y[0]

'metal'

In [162]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
y = le.fit_transform(y)

In [163]:
y_temp = keras.utils.to_categorical(y,num_classes=10)
print(X.shape)

(1000, 84)


In [165]:
import keras
from keras.layers import Dense, add
from keras.models import Sequential
from keras import optimizers
import keras.utils
from sklearn.model_selection import KFold


kf = KFold(n_splits=5)
kf.get_n_splits(X)

print(kf)  
def create_model():
    model = Sequential()
    model.add(Dense(56,input_shape=(84,)))
    model.add(Dense(52,activation='relu'))
    model.add(Dense(32,activation='relu'))
    model.add(Dense(16,activation='relu'))
    model.add(Dense(10,activation='softmax'))
    model.compile(optimizers.Adam(lr=0.005),loss='categorical_crossentropy',metrics=['accuracy'])
    return model
for train_index, test_index in kf.split(X):
   #print("TRAIN:", train_index, "TEST:", test_index)
   X_train, X_test = X[train_index], X[test_index]
   y_train, y_test = y_temp[train_index], y_temp[test_index]
   clf = create_model()
   clf.fit(X_train, y_train, epochs=200, batch_size=32)
   print(clf.evaluate(X_test,y_test))

KFold(n_splits=5, random_state=None, shuffle=False)


Epoch 1/200


 32/800 [>.............................] - ETA: 33s - loss: 2.3675 - acc: 0.0625

768/800 [===========================>..] - ETA: 0s - loss: 2.1197 - acc: 0.1862 

800/800 [==============================] - 1s 2ms/step - loss: 2.1100 - acc: 0.1913


...

Epoch 12/200
 32/800 [>.............................] - ETA: 0s - loss: 0.5117 - acc: 0.8438

768/800 [===========================>..] - ETA: 0s - loss: 0.6519 - acc: 0.7552

800/800 [==============================] - 0s 71us/step - loss: 0.6572 - acc: 0.7538


Epoch 13/200
 32/800 [>.............................] - ETA: 0s - loss: 0.4203 - acc: 0.8750

KeyboardInterrupt: 

ValueError: invalid literal for int() with base 10: 'metal'